<a href='https://jupyter.org/'>Jupyter Notebook 2</a>

## Multi-effect crystallizer model

### Diagram of a 4-effect evaporative crystallizer exteded from WaterTAP crystallizer model. 
##### (The optimized operating temperature and pressure of each effect for the produced water case are labelled.)
<p align="center">
  <img src="multieffect_cryst.png" width="80%">
</p>

#### Import packages

In [35]:
from watertap_contrib.reflo.analysis.case_studies.KBHDP.components.multi_effect_crystallizer import (
    build_mec,
    build_system,
    init_system,
    display_mec_streams,
    set_mec_scaling,
    add_mec_costing,
    get_model_performance,
    set_system_operating_conditions,
    display_mec_dof,
)

from idaes.core.solvers import get_solver
from pyomo.environ import (
    assert_optimal_termination
)

#### Create a feed flow of crystallizer with 0.36 MGD and 127 g/L TDS

In [36]:
solver = get_solver()
m = build_system()
blk = m.fs.MEC

set_system_operating_conditions(m, blk, Qin=0.35587876, tds=127)
init_system(m, blk)

results = solver.solve(m)
assert_optimal_termination(results)


2025-01-27 19:29:15 [INFO] idaes.init.fs.feed: Initialization Complete.
2025-01-27 19:29:15 [INFO] idaes.init.fs.feed: Initialization Complete.
2025-01-27 19:29:15 [INFO] idaes.init.fs.MEC.unit.effects[1].effect.heating_steam: fs.MEC.unit.effects[1].effect.heating_steam State Released.
2025-01-27 19:29:15 [WARNING] idaes.watertap.property_models.water_prop_pack: While using the calculate_state method on fs.MEC.unit.effects[1].effect.heating_steam[0.0], variable flow_mass_phase_comp was provided as an argument in var_args, but it does not have a scaling factor. This suggests that the calculate_scaling_factor method has not been used or the variable was created on demand after the scaling factors were calculated. It is recommended to touch all relevant variables (i.e. call them or set an initial value) before using the calculate_scaling_factor method.
2025-01-27 19:29:15 [WARNING] idaes.watertap.property_models.water_prop_pack: While using the calculate_state method on fs.MEC.unit.effect

#### Retreive the overall performance and individual performance for each effect

In [37]:
data_table, overall_performance = get_model_performance(m)

In [38]:
for i,v in overall_performance.items():
    print(f'{i}: {v}')

Capacity (m3/day): 1405.5841276680067
Feed brine salinity (g/L): 121.7200368218603
Total brine disposed (kg/s): 17.572169022630547
Total water production (kg/s): 12.950526092920779
Total solids collected (kg/s): 0.990090996110949
Total waste water remained (kg/s): 3.6315516135988184
Initial thermal energy consumption (kW): 9462.819985918708
Overall STEC (kWh/m3 feed): 161.57530182049047
Total heat transfer area (m2): 852.4583216787306


In [39]:
print(data_table)

                                             Effect 1     Effect 2  \
Feed mass flow rate (kg/s)                   4.931900     4.499194   
Feed volumetric flow rate (L/s)              4.565959     4.165359   
Feed salinities (g/L)                      121.720037   121.720037   
Operating temperature (C)                   86.332541    71.713554   
Vapor condensation temperature (C)          78.776244    64.835190   
Operating pressure (bar)                     0.450000     0.250000   
Water production (kg/s)                      3.648270     3.316679   
Solid production (kg/s)                      0.277884     0.253504   
Liquid waste (kg/s)                          1.005746     0.929011   
Liquid waste volumetric flow rate (L/s)      0.857434     0.788587   
Liquid waste salinity (g/L)                 10.000000    10.000000   
Thermal energy requirement (kW)           9462.819986  8477.880082   
Thermal energy available from vapor (kW)  8477.880082  7819.850693   
Vapor enthalpy (kJ) 

### Optimization Scenario
 Object: Minimize heat exchangers area

 Constraint 1:  Operating pressure/temperature is decreasing across the effects
 
 Constraint 2:  The minimnum temperature difference between effects is 12 degree C

In [40]:
from pyomo.environ import Constraint, Objective, TerminationCondition

effs = {i: m.fs.MEC.unit.effects[i].effect for i in m.fs.MEC.unit.Effects}
# Unfix variables and set lower bound of the last effect
effs[1].pressure_operating.unfix()
effs[2].pressure_operating.unfix()
effs[3].pressure_operating.unfix()
effs[4].pressure_operating.unfix()
effs[4].pressure_operating.setlb(0.2 * 1e5)


# Add constraint 1
@m.Constraint(m.fs.MEC.unit.Effects,
              doc="Pressure decreasing")
def pressure_bound(b,j):
    if j <4:
        return effs[j+1].pressure_operating <= effs[j].pressure_operating
    else:
        return Constraint.Skip

# Add constraint 2
@m.Constraint(m.fs.MEC.unit.Effects,
              doc="Temperature difference")
def temp_bound(b,j):
    if j < 4:
        return (
            effs[j+1].temperature_operating
            >= effs[j].temperature_operating - 12
        )
    else:
        return Constraint.Skip

total_area = sum(effs[i].heat_exchanger_area for i in m.fs.MEC.unit.Effects)

# Optimize the heat transfer area
m.fs.objective = Objective(expr=total_area)


optimization_results = solver.solve(m, tee=False)
assert (
    optimization_results.solver.termination_condition
    == TerminationCondition.optimal
)
data_table2, overall_performance2 = get_model_performance(m)

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.


#### Derive new operational point based on the optimization results

In [41]:
print(data_table2)

                                              Effect 1     Effect 2  \
Feed mass flow rate (kg/s)                    5.147810     4.542344   
Feed volumetric flow rate (L/s)               4.765849     4.205307   
Feed salinities (g/L)                       121.720037   121.720037   
Operating temperature (C)                   100.000004    90.176230   
Vapor condensation temperature (C)           91.759108    82.434266   
Operating pressure (bar)                      0.747236     0.521112   
Water production (kg/s)                       3.821554     3.363366   
Solid production (kg/s)                       0.290050     0.255935   
Liquid waste (kg/s)                           1.036206     0.923042   
Liquid waste volumetric flow rate (L/s)       0.886753     0.787784   
Liquid waste salinity (g/L)                 327.091700   324.879833   
Thermal energy requirement (kW)           10041.332265  8756.285544   
Thermal energy available from vapor (kW)   8756.285544  7785.329590   
Vapor 